In [ ]:
# 多行输出结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# BeautifulSoup的使用

## 0. 基本使用

In [28]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

In [14]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[:100])

<html>
 <body>
  <div class="panel">
   <div class="panel-body">
    <a>
     Hello, this is a link



首先我们声明了一个变量 `html`，它是一个 `HTML` 字符串，但是注意到，它并不是一个完整的 `HTML` 字符串，`body` 和 `html` 节点都没有闭合，但是我们将它当作第一个参数传给 `BeautifulSoup` 对象，第二个参数传入的是解析器的类型，在这里我们使用 `lxml`，这样就完成了 `BeaufulSoup` 对象的初始化，将它赋值给 `soup` 这个变量。  
那么接下来我们就可以通过调用 `soup` 的各个方法和属性对这串 `HTML` 代码解析了。
我们首先调用了 `prettify()` 方法，这个方法可以把要解析的字符串以标准的缩进格式输出，在这里注意到输出结果里面包含了 `body` 和 `html` 节点，也就是说对于不标准的 `HTML` 字符串 `BeautifulSoup` 可以自动更正格式，这一步实际上不是由 `prettify()` 方法做的，这个更正实际上在初始化 `BeautifulSoup` 时就完成了。  
然后我们调用了 `soup.title.string` ，这个实际上是输出了 `HTML` 中 `title` 节点的文本内容。所以 `soup.title` 就可以选择出 `HTML` 中的 `title` 节点，再调用 `string` 属性就可以得到里面的文本了，所以我们就可以通过简单地调用几个属性就可以完成文本的提取了

## 1. 节点选择器
选择元素的时候直接通过调用节点的名称就可以选择节点元素了，然后再调用 `string` 属性就可以得到节点内的文本

### 1.1 选择元素

In [29]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'lxml')
print('soup.title: ', soup.title)
print('type(soup.title): ', type(soup.title))
print('soup.title.string: ', soup.title.string)

print('soup.head: ', soup.head)
print('soup.p: ', soup.p)

soup.title:  <title>The Dormouse's story</title>
type(soup.title):  <class 'bs4.element.Tag'>
soup.title.string:  The Dormouse's story
soup.head:  <head><title>The Dormouse's story</title></head>
soup.p:  <p class="title" name="dromouse"><b>The Dormouse's story</b></p>


### 1.2 提取信息
调用 `string` 属性来获取文本的值，那我们要获取节点属性值怎么办呢？获取节点名怎么办呢？下面我们来统一梳理一下信息的提取方式

#### 1.2.1 获取名称

In [30]:
print('soup.title.name: ', soup.title.name)

soup.title.name:  title


#### 1.2.2 获取属性 `.attrs` 或者 字典调用方式

In [31]:
print('soup.p.attrs: ', soup.p.attrs)
print('soup.p.attrs["name"]: ', soup.p.attrs['name'])

soup.p.attrs:  {'class': ['title'], 'name': 'dromouse'}
soup.p.attrs["name"]:  dromouse


可以看到 `attrs` 的返回结果是字典形式，把选择的节点的所有属性和属性值组合成一个字典，接下来如果要获取 `name` 属性，就相当于从字典中获取某个键值，只需要用中括号加属性名称就可以得到结果了，比如获取 `name` 属性就可以通过 `attrs['name']` 得到相应的属性值。

In [32]:
print("soup.p['name']: ", soup.p['name'])
print("soup.p['class']: ", soup.p['class'])

soup.p['name']:  dromouse
soup.p['class']:  ['title']


#### 1.2.3 获取内容  `.string` 属性 或者 `get_text()` 方法

In [33]:
print('soup.p.string: ', soup.p.string)
print('soup.p.get_text(): ', soup.p.get_text())

soup.p.string:  The Dormouse's story
soup.p.get_text():  The Dormouse's story


#### 1.3 嵌套选择
在上面的例子中我们知道每一个返回结果都是 `bs4.element.Tag` 类型，它同样可以继续调用节点进行下一步的选择，比如我们获取了 `head` 节点元素，我们可以继续调用 `head` 来选取其内部的 `head` 节点元素。

In [14]:
print(soup.head)
print(soup.head.title)
print(soup.head.title.string)
print(soup.head.string)

<head><title>The Dormouse's story</title></head>
<title>The Dormouse's story</title>
The Dormouse's story
The Dormouse's story


### 1.4 关联选择
有时候在选择的时候需要先选中某一个节点元素，然后以它为基准再选择它的子节点、父节点、兄弟节点等等。

1. 子节点和子孙节点
  - 子节点: 调用 `contents` 属性或者 `children` 属性
  - 子孙节点: 调用 `descendants` 属性, 返回生成器, 可以用 `list()` 方法转化 
2. 父节点和祖先节点
  - 父节点: 调用 `parent` 属性
  - 祖先节点: 调用 `parents` 属性, 返回生成器, 可以用 `list()` 方法转化
3. 兄弟节点
  - 上一个同级: 调用 `previous_sibling` 属性
  - 下一个同级: 调用 `next_sibling` 属性
  - 上面所有同级: 调用 `previous_siblings` 属性
  - 下面所有同级: 调用 `next_siblings` 属性

## 2. 方法选择器

`BeautifulSoup` 还提供了一些查询的方法，比如 `find_all()`、`find()` 等方法，调用方法然后传入相应等参数就可以灵活地进行查询了。  
最常用的查询方法有 `find_all()` 和 `find()` 

### 2.1 find_all()

返回的是所有匹配的元素组成的列表

api: `find_all(name, attrs, recursive, text, **kwargs)`

#### 2.1.1 name
根据节点名来查询元素

In [4]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1" name="elements">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'lxml')

In [25]:
print("soup.find_all(name='ul'): ", soup.find_all(name='ul'))
print("type(soup.find_all(name='ul')[0]): ", type(soup.find_all(name='ul')[0]))

soup.find_all(name='ul'):  [<ul class="list" id="list-1" name="elements">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>, <ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>]
type(soup.find_all(name='ul')[0]):  <class 'bs4.element.Tag'>


再进行嵌套遍历

In [26]:
for ul in soup.find_all(name='ul'):
    lis = ul.find_all(name='li')
    print(lis)
    for li in lis:
        print(li.string)

[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
Foo
Bar
Jay
[<li class="element">Foo</li>, <li class="element">Bar</li>]
Foo
Bar


#### 2.1.2 attrs
传入属性来进行查询

In [6]:
soup = BeautifulSoup(html, 'lxml')
print(soup.find_all(attrs={'id': 'list-1'}))
print(soup.find_all(attrs={'name': 'elements'}))

[<ul class="list" id="list-1" name="elements">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<ul class="list" id="list-1" name="elements">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]


对于一些常用的属性比如 `id` 、`class` 等(没有 `name`)，我们可以不用 `attrs` 来传递，比如我们要查询 `id` 为 `list-1` 的节点，我们可以直接传入 `id` 这个参数，还是上面的文本，我们换一种方式来查询。

In [8]:
print(soup.find_all(id='list-1'))
print(soup.find_all(class_='element'))

[<ul class="list" id="list-1" name="elements">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]


#### 2.1.3 text
text 参数可以用来匹配节点的文本，传入的形式可以是字符串，可以是正则表达式对象

In [13]:
import re
html='''
<div class="panel">
    <div class="panel-body">
        <a>Hello, this is a link</a>
        <a>Hello, this is a link, too</a>
    </div>
</div>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.find_all(text=re.compile('link')))

['Hello, this is a link', 'Hello, this is a link, too']


在这里有两个 `a` 节点，其内部包含有文本信息，在这里我们调用 `find_all()` 方法传入 `text` 参数，参数为正则表达式对象，结果会返回所有匹配正则表达式的节点文本组成的列表。

### 2.2 方法总结
- `find()` 返回单个元素，也就是第一个匹配的元素,  
  `find_all()` 返回所有符合条件的节点
- `find_parent()` 返回直接父节点,  
  `find_parents()` 返回所有祖先节点
- `find_next_sibling()` 返回后面第一个兄弟节点,  
  `find_next_siblings()` 返回后面所有兄弟节点
- `find_previous_sibling()` 返回前面第一个兄弟节点,  
  `find_previous_siblings()` 返回前面所有兄弟节点
- `find_next()` 返回第一个符合条件的节点,  
  `find_all_next()` 返回节点后所有符合条件的节点
- `find_previous()` 返回第一个符合条件的节点,  
  `find_all_previous()` 返回节点后所有符合条件的节点

## 3. CSS选择器
BeautifulSoup 还提供了另外一种选择器 -- `CSS` 选择器, 调用 select() 方法使用。  
CSS选择器完整参考手册http://www.w3school.com.cn/cssref/css_selectors.asp
常用选择器:  

| 选择器 | 例子 | 例子描述 |
|-------|-----|---------|
| `.class` | `.intro` | 选择 `class="intro"` 的所有元素. |
| `#id` | `#firstname` | 选择 `id="firstname"` 的所有元素. |
| `*` | `*` | 选择所有元素. |
| `element` | `p` | 选择所有 `<p>` 元素. |
| `element,element` | `div,p` | 选择所有 `<div>` 元素和所有 `<p>` 元素. |
| `element element` | `div p` | 选择 `<div>` 元素内部的所有 `<p>` 元素. |
| `element>element` | `div>p` | 选择父元素为 `<div>` 元素的所有 `<p>` 元素. |
| `element+element` | `div+p` | 选择紧接在 `<div>` 元素之后的所有 `<p>` 元素. |
| `[attribute]` | `[target]` | 选择带有 target 属性所有元素. |
| `[attribute=value]` | `[target=_blank]` | 选择 `target="_blank"` 的所有元素. |
| `[attribute~=value]` | `[title~=flower]` | 选择 `title` 属性包含单词 `"flower"` 的所有元素. |
| <code>[attribute&#124;=value]</code> | <code>[lang&#124;=en]</code> | 选择 `lang` 属性值以 `"en"` 开头的所有元素. |
| `:lang(language)` | `p:lang(it)` | 选择带有以 `"it"` 开头的 `lang` 属性值的每个 `<p>` 元素. |
| `element1~element2` | `p~ul` | 选择前面有 `<p>` 元素的每个 `<ul>` 元素. |
| `[attribute^=value]` | `a[src^="https"]` | 选择其 `src` 属性值以 `"https"` 开头的每个 `<a>` 元素. |
| `[attribute竖线=value]` | `a[src竖线=".pdf"]` | 选择其 `src` 属性以 `".pdf"` 结尾的所有 `<a>` 元素. |
| `[attribute*=value]` | `a[src*="abc"]` | 选择其 `src` 属性中包含 `"abc"` 子串的每个 `<a>` 元素. |
| `:first-of-type` | `p:first-of-type` | 选择属于其父元素的首个 `<p>` 元素的每个 `<p>` 元素. |
| `:last-of-type` | `p:last-of-type` | 选择属于其父元素的最后 `<p>` 元素的每个 `<p>` 元素. |
| `:only-of-type` | `p:only-of-type` | 选择属于其父元素唯一的 `<p>` 元素的每个 `<p>` 元素. |
| `:only-child` | `p:only-child` | 选择属于其父元素的唯一子元素的每个 `<p>` 元素. |
| `:nth-child(n)` | `p:nth-child(2)` | 选择属于其父元素的第二个子元素的每个 `<p>` 元素. |
| `:nth-last-child(n)` | `p:nth-last-child(2)` | 同上，从最后一个子元素开始计数. |

### 3.1 基本使用

In [46]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''

In [47]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'lxml')
print(soup.select('.panel .panel-heading'))
print(soup.select('ul li'))
print(soup.select('#list-2 .element'))
print(type(soup.select('ul')[0]))

[<div class="panel-heading">
<h4>Hello</h4>
</div>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]
<class 'bs4.element.Tag'>


### 3.2 嵌套选择
`select()` 方法同样支持嵌套选择，例如我们先选择所有 `ul` 节点，再遍历每个 `ul` 节点选择其 `li` 节点

In [26]:
soup = BeautifulSoup(html, 'lxml')
for ul in soup.select('ul'):
    print(ul.select('li'))
type(ul)

[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]


bs4.element.Tag

### 3.3 获取属性
由上可知节点类型还是 `Tag` ，所有获取属性还是可以使用上面 节点选择器1.2.2 的方法

In [35]:
soup = BeautifulSoup(html, 'lxml')
for ul in soup.select('ul'):
    print(ul.attrs['id'])
    print(ul['id'])

list-1
list-1
list-2
list-2


### 3.4 获取文本
获取文本还是可以使用 节点选择器1.2.3 的方法: `.string`属性 或者 `get_text()` 方法

In [50]:
for li in soup.select('li'):
    print('get_text(): ', li.get_text())
    print('.string: ', li.string)

get_text():  Foo
.string:  Foo
get_text():  Bar
.string:  Bar
get_text():  Jay
.string:  Jay
get_text():  Foo
.string:  Foo
get_text():  Bar
.string:  Bar


## 4. 例子：
使用 `bs4` 库的 `Beautifulsoup` 解析猫眼电影排行网页

In [48]:
import requests

url = 'http://maoyan.com/board/4?offset=0'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'
}

try:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print('获取网页成功!')
except RequestException as e:
    print(e)

获取网页成功!


### 4.1 节点选择器解析

In [31]:
from bs4 import BeautifulSoup

# response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')
dd = soup.find_all(name='dd')

results = []
for item in dd:
    result = {
        'rank': item.find('i').string,
        'img_url': item.find('img', class_='board-img')['data-src'],
        'name': item.find('p', class_='name').string,
        'actor': item.find('p', class_='star').string.strip(),
        'time': item.find('p', class_='releasetime').string,
        'score': item.find('i', class_='integer').string +\
            item.find('i', class_='fraction').string
    }
    results.append(result)

print(results[:2])

[{'rank': '1', 'img_url': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'name': '霸王别姬', 'actor': '主演：张国荣,张丰毅,巩俐', 'time': '上映时间：1993-01-01(中国香港)', 'score': '9.6'}, {'rank': '2', 'img_url': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'name': '罗马假日', 'actor': '主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '上映时间：1953-09-02(美国)', 'score': '9.1'}]


### 4.2 CSS选择器解析

In [59]:
from bs4 import BeautifulSoup

# response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')
dd = soup.select('dl dd')

results = []
for item in dd:
    result = {
        'rank': item.select('i')[0].string,
        'img_url': item.select('img')[1]['data-src'],
        'name': item.select('.name')[0].string,
        'actor': item.select('.star')[0].string.strip(),
        'time': item.select('.releasetime')[0].string,
        'score': item.select('.integer')[0].string +\
            item.select('.fraction')[0].string
    }
    results.append(result)

print(results[:2])

[{'rank': '1', 'img_url': 'http://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'name': '霸王别姬', 'actor': '主演：张国荣,张丰毅,巩俐', 'time': '上映时间：1993-01-01(中国香港)', 'score': '9.6'}, {'rank': '2', 'img_url': 'http://p0.meituan.net/movie/54617769d96807e4d81804284ffe2a27239007.jpg@160w_220h_1e_1c', 'name': '罗马假日', 'actor': '主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '上映时间：1953-09-02(美国)', 'score': '9.1'}]
